In [1]:
import BioSimSpace as bss
import glob
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

INFO:rdkit:Enabling RDKit 2022.09.1 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [4]:
paths = sorted(glob.glob("/home/jguven/projects/metalloenzymes/vim_2/outputs/SOMD_*/*/"))
ligands = [path.split("/")[-2] for path in paths]

stages = ["bound", "unbound"]

for stage in stages:
    for i in range(len(paths)):
        stage_path = paths[i] + stage
        mbar_file = stage_path + "/mbar.txt"

        start_index = 1
        end_index = -1
        matrix_lines = []
        with open(mbar_file) as file:
            lines = file.readlines()
            for i in range(len(lines)):
                if "#Overlap matrix" in lines[i]:
                    start_index = i + 1
                elif "#DG from neighbouring lambda in kcal/mol" in lines[i]:
                    end_index = i
            matrix_lines = lines[start_index:end_index]
        
        matrix = []
        for line in matrix_lines:
            split_line = line.replace("\n", "").split(" ")
            new_line = [float(x) for x in split_line]
            matrix.append(new_line)
        
        colour_map = matplotlib.colors.ListedColormap(['#FBE8EB','#68246D','#61BF1A', '#154734'])
        n_colours = colour_map.N
        boundary_values = [0.0, 0.025, 0.1, 0.3, 0.8]
        norm_colours = matplotlib.colors.BoundaryNorm(boundary_values, n_colours, clip=False)
        colour_bar_args = dict(ticks=[0.025, 0.1, 0.3, 0.8],
                            shrink=0.685)

        fig = plt.figure(figsize=(8, 8))
        sns.set_style("ticks")
        sns.set_context("notebook", font_scale=2)
        ax = sns.heatmap(matrix, 
                        annot=True, 
                        fmt=".2f", 
                        linewidths=0.3, 
                        annot_kws={"size": 14}, 
                        square=True, 
                        robust=True, 
                        cmap=colour_map,
                        norm=norm_colours, 
                        cbar_kws=colour_bar_args,
                        vmax=1
                        )
        ax.xaxis.tick_top()
        ax.tick_params(axis="y", rotation=360)
        ax.set_title(r"$\lambda$ index", fontsize=24)
        ax.set_ylabel(r"$\lambda$ index", fontsize=24)
        # ax.set_xlabel(f"{stage} ligand {ligand}", fontsize=24)
        fig.tight_layout()
        plt.savefig(f"/home/jguven/projects/metalloenzymes/vim_2/plots/om_{stage}_{ligands[i]}.png", transparent=True)